# 99. 자료 출처
## 99-1. 블로그
- 위저원의 블로그 : Build a Steam Game Recomend Model (1/?) - 데이터 수집 경로 설정하기 | [[블로그 링크]](https://weejw.tistory.com/555)

In [3]:
from multiprocessing import Pool, Process
import pandas as pd
import requests
import os, re
import time
import json

In [14]:
def return_or_print(response):
    
    if response.status_code == 200: return response.json()
    else: print(f'no response data with code : {response.status_code}')

    
get_api  = lambda url: return_or_print(requests.get(url))

def get_info(appid):
    
    response = requests.get(f'https://store.steampowered.com/api/appdetails?appids={appid}&l=korean')
    return return_or_print(response)
    
    
def get_reviews(appid):
    
    response = requests.get(f'https://store.steampowered.com/appreviews/{appid}?json=1')
    return return_or_print(response)


def get_requirements(info, platform = 'pc', typeof = 'minimum'):
    
    assert platform in ['pc', 'linux', 'mac'], 'platform must be pc, linux and mac'
    assert typeof in ['minimum', 'recommended'], 'type of must be minimum, recommended'
    
    requirements = info[f'{platform}_requirements'][typeof]
    requirements = re.sub('(<([^>]+)>)', '\n', requirements)
    
    requirements = requirements.split('\n')
    requirements = [req for req in requirements if req != '']
    
    result = {}
    for idx in range(1, len(requirements), 2): 
        
        print(requirements[idx: idx + 2])
        key, value = requirements[idx: idx + 2]
        result[key] = value
        
    return result

In [5]:
steam_games = get_api('https://api.steampowered.com/ISteamApps/GetAppList/v2')['applist']['apps']

In [6]:
appid = steam_games[-1]['appid']
info  = get_info(appid)[str(appid)]['data']

In [7]:
name, typeof      = info['name'], info['type']
appid, is_free    = info['steam_appid'], info['is_free']
short_description = info['short_description']
languages         = info['supported_languages']
image             = info['header_image']
web_site          = info['website']
platforms         = info['platforms']
release_date      = info['release_date']

In [9]:
info['pc_requirements']

{'minimum': '<strong>최소:</strong><br><ul class="bb_ul"><li>64비트 프로세서와 운영 체제가 필요합니다<br></li><li><strong>운영체제:</strong> Windows 7<br></li><li><strong>프로세서:</strong> Intel Core i5-2300 | AMD FX-4350<br></li><li><strong>메모리:</strong> 8 GB RAM<br></li><li><strong>그래픽:</strong> Nvidia GeForce GTS 450, 1 GB | AMD Radeon HD 5770, 1 GB<br></li><li><strong>DirectX:</strong> 버전 11<br></li><li><strong>저장공간:</strong> 15 GB 사용 가능 공간</li></ul>',
 'recommended': '<strong>권장:</strong><br><ul class="bb_ul"><li>64비트 프로세서와 운영 체제가 필요합니다</li></ul>'}

In [15]:
win_minimum = get_requirements(info, 'pc')
win_minimum

['64비트 프로세서와 운영 체제가 필요합니다', '운영체제:']
[' Windows 7', '프로세서:']
[' Intel Core i5-2300 | AMD FX-4350', '메모리:']
[' 8 GB RAM', '그래픽:']
[' Nvidia GeForce GTS 450, 1 GB | AMD Radeon HD 5770, 1 GB', 'DirectX:']
[' 버전 11', '저장공간:']
[' 15 GB 사용 가능 공간']


ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
win_recommen = get_requirements(info, 'pc', 'recommended')
win_recommen

In [34]:
def save_json(idx, games):
    
    database = {}
    
    for game in games:
        appid     = game['appid']
        
        try:
            json_path = f'jsons/{appid}' 
            
            try:
                info = get_info(appid)[str(appid)]['data']
                
                os.makedirs(json_path, exist_ok = True)
                json.dump(info, open(f'{json_path}/{appid}.json', 'w'), indent = 4)
            
            except Exception as e:
                print(f'[Error. 001] json file can\'t save : [{appid}] {e}\n')
                
            database['appid']     = appid
            database['json_path'] = f'{json_path}/{appid}.json'
        
        except Exception as e:
            print(f'[Error. 002] api data receieve failed : [{appid}] {e}\n')
        time.sleep(5)
        
    print(f'[Done] process : {idx} done.\n')

In [35]:
_1st_jobs = steam_games[:50]
_2nd_jobs = steam_games[50 : 100]
_3rd_jobs = steam_games[100 : 150]
_4th_jobs = steam_games[150 : 200]

jobs = [_1st_jobs, _2nd_jobs, _3rd_jobs, _4th_jobs]
len(_1st_jobs), len(_2nd_jobs), len(_3rd_jobs), len(_4th_jobs)

(50, 50, 50, 50)

In [36]:
!rm -rf jsons

In [37]:
start = time.time()
procs = []

for idx, job in enumerate(jobs, 1):
    p = Process(target = save_json, args = (idx, job, ))
    p.start()
    procs.append(p)
    
for p in procs: p.join()

f'[Done] 수행 시간 : {time.time() - start:.4f}초'

[Error. 001] json file can't save : [2107760] 'data'
[Error. 001] json file can't save : [1941401] 'data'


[Error. 001] json file can't save : [2170321] 'data'

[Error. 001] json file can't save : [1825161] 'data'

[Error. 001] json file can't save : [1897482] 'data'

[Error. 001] json file can't save : [2112761] 'data'

[Error. 001] json file can't save : [1829051] 'data'

[Error. 001] json file can't save : [1808781] 'data'

[Error. 001] json file can't save : [1983382] 'data'

[Error. 001] json file can't save : [1828741] 'data'

[Error. 001] json file can't save : [216938] 'data'

[Error. 001] json file can't save : [660010] 'data'

[Error. 001] json file can't save : [660130] 'data'

[Error. 001] json file can't save : [1118314] 'data'

[Error. 001] json file can't save : [1275822] 'data'

[Error. 001] json file can't save : [2107121] 'data'

[Error. 001] json file can't save : [1343832] 'data'

[Error. 001] json file can't save : [2107123] 'data'

[Error. 001] json file can't sa

'[Done] 수행 시간 : 267.9750초'

In [38]:
database

{}